# modify_tensor vs. create_tensor
- modify_tensor 는 Tensor를 입력으로 받아서 어떤 수정이나 변경을 함
- create_tensor 는 Tensor 입력 없이 tensor를 만드는 것들

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.15.2'

In [4]:
sess = tf.Session()

## constant

In [3]:
tf.constant([1,2,3])

<tf.Tensor 'Const:0' shape=(3,) dtype=int32>

## convert_to_tensor

https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/convert_to_tensor

In [5]:
import numpy as np

In [6]:
tgt = np.ndarray(shape=(1,2,1,1,3), dtype=float)
tsr = tf.convert_to_tensor(tgt)
tsr

<tf.Tensor 'Const_1:0' shape=(1, 2, 1, 1, 3) dtype=float64>

### iterable -> tensor operation 예

In [7]:
import numpy as np

def my_func(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    return tf.matmul(arg, arg) + arg

# The following calls are equivalent.
value_1 = my_func(tf.constant([[1.0, 2.0], [3.0, 4.0]]))
value_2 = my_func([[1.0, 2.0], [3.0, 4.0]])
value_3 = my_func(np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32))


## Variable
선언하고, initializer 도 실행해줘야 한다.

### initial_value 는 필요하다

In [145]:
tf.Variable(
    initial_value=[1,2,3],
    trainable=None,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None,
    use_resource=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.VariableAggregation.NONE,
    shape=None
)

<tf.Variable 'Variable_2:0' shape=(3,) dtype=int32_ref>

### initializing 이 필요하다

In [146]:
tsr = tf.Variable([1,2,3])

sess.run(tsr)

FailedPreconditionError: Attempting to use uninitialized value Variable_3
	 [[{{node _retval_Variable_3_0_0}}]]

In [147]:
tsr = tf.Variable([1,2,3])

sess.run(tsr.initializer)
sess.run(tsr)

array([1, 2, 3], dtype=int32)

## assign

https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/assign

### Variable 에 대해만 동작한다

In [12]:
tgt = tf.Variable([1,2,3])
res = tf.assign(tgt, tf.constant([4,5,6]))
sess.run(res)

array([4, 5, 6], dtype=int32)

### Constant node 에 대해서는 에러

In [13]:
tgt = tf.constant([1,2,3])
res = tf.assign(tgt, tf.constant([4,5,6]))
sess.run(res)

AttributeError: 'Tensor' object has no attribute 'assign'

# numpy-like methods

## ones

In [14]:
tf.ones([2, 3], tf.int32)

<tf.Tensor 'ones:0' shape=(2, 3) dtype=int32>

### shape must be rank 1

In [22]:
tf.ones([[2,3],[2,3]], tf.int32)

ValueError: Shape must be rank 1 but is rank 2 for 'ones_4' (op: 'Fill') with input shapes: [2,2], [].

## ones_like

In [15]:
tensor = tf.constant([[1, 2, 3], [4, 5, 6]])
tf.ones_like(tensor)  # [[1, 1, 1], [1, 1, 1]]

<tf.Tensor 'ones_like:0' shape=(2, 3) dtype=int32>

## eye

In [39]:
res = tf.eye(2)

sess.run(res)

array([[1., 0.],
       [0., 1.]], dtype=float32)

### batch_shape 을 통해 eye 를 묶어서 만든다

In [42]:
batch_identity = tf.eye(2, batch_shape=[3])

sess.run(batch_identity)

array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]], dtype=float32)

In [43]:
batch_identity = tf.eye(2, batch_shape=[3,3,1])

print(sess.run(batch_identity))
print(batch_identity.shape)

[[[[[1. 0.]
    [0. 1.]]]


  [[[1. 0.]
    [0. 1.]]]


  [[[1. 0.]
    [0. 1.]]]]



 [[[[1. 0.]
    [0. 1.]]]


  [[[1. 0.]
    [0. 1.]]]


  [[[1. 0.]
    [0. 1.]]]]



 [[[[1. 0.]
    [0. 1.]]]


  [[[1. 0.]
    [0. 1.]]]


  [[[1. 0.]
    [0. 1.]]]]]
(3, 3, 1, 2, 2)


### num_columns 와 row_columns 를 이용할 수 있다

In [44]:
res = tf.eye(2, num_columns=3)

print(sess.run(res))
print(res.shape)

[[1. 0. 0.]
 [0. 1. 0.]]
(2, 3)


In [47]:
res = tf.eye(num_rows=2)

print(sess.run(res))
print(res.shape)

[[1. 0.]
 [0. 1.]]
(2, 2)


In [48]:
res = tf.eye(num_rows=2, num_columns=3)

print(sess.run(res))
print(res.shape)

[[1. 0. 0.]
 [0. 1. 0.]]
(2, 3)


# convinient tool

## one_hot

### depth = 3 과 indices 에 따라 만든다

In [16]:
indices = [0, 1, 2]
depth = 3
output = tf.one_hot(indices, depth)  # output: [3 x 3]

sess.run(output)
# [[1., 0., 0.],
#  [0., 1., 0.],
#  [0., 0., 1.]]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

### on_value 과 off_value 를 정해줄 수 있다

In [34]:
indices = [0, 2, -1, 1]
depth = 3
output = tf.one_hot(indices, depth,
           on_value=5.0, off_value=0.0,
           axis=-1)  # output: [4 x 3]

sess.run(output)
# [[5.0, 0.0, 0.0],  # one_hot(0)
#  [0.0, 0.0, 5.0],  # one_hot(2)
#  [0.0, 0.0, 0.0],  # one_hot(-1)
#  [0.0, 5.0, 0.0]]  # one_hot(1)


array([[5., 0., 0.],
       [0., 0., 5.],
       [0., 0., 0.],
       [0., 5., 0.]], dtype=float32)

### indices 의 rank 가 높아지면

In [18]:
indices = [[0, 2], [1, -1]]
depth = 3
output = tf.one_hot(indices, depth,
           on_value=1.0, off_value=0.0,
           axis=-1)  # output: [2 x 2 x 3]

sess.run(output)
# [[[1.0, 0.0, 0.0],   # one_hot(0)
#   [0.0, 0.0, 1.0]],  # one_hot(2)
#  [[0.0, 1.0, 0.0],   # one_hot(1)
#   [0.0, 0.0, 0.0]]]  # one_hot(-1)


array([[[1., 0., 0.],
        [0., 0., 1.]],

       [[0., 1., 0.],
        [0., 0., 0.]]], dtype=float32)

In [37]:
indices = [[[0, 2], [1, -1]], [[2, 0], [-1, 1]]]
depth = 3
output = tf.one_hot(indices, depth,
           on_value=1.0, off_value=0.0,
           axis=-1)  # output: [2 x 2 x 2 x 3]

print(sess.run(output))
print(output.shape)

[[[[1. 0. 0.]
   [0. 0. 1.]]

  [[0. 1. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 1.]
   [1. 0. 0.]]

  [[0. 0. 0.]
   [0. 1. 0.]]]]
(2, 2, 2, 3)


### axis=-1 가 default 이다

In [33]:
indices = [0, 1, 2]
depth = 4
output = tf.one_hot(indices, depth, axis=-1)  # output: [3 x 4]

sess.run(output)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [31]:
indices = [0, 1, 2]
depth = 4
output = tf.one_hot(indices, depth, axis=0)  # output: [4 x 3]

sess.run(output)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)

In [32]:
indices = [0, 1, 2]
depth = 4
output = tf.one_hot(indices, depth)  # output: [3 x 4]

sess.run(output)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

## fill

In [49]:
res = tf.fill([2, 3], 9)

sess.run(res)

array([[9, 9, 9],
       [9, 9, 9]], dtype=int32)

### constant 와 비교

- tf.fill only supports scalar contents, whereas tf.constant supports Tensor values.
- tf.fill creates an Op in the computation graph that constructs the actual Tensor value at runtime. This is in contrast to tf.constant which embeds the entire Tensor into the graph with a Const node.
- Because tf.fill evaluates at graph runtime, it supports **dynamic shapes based on other runtime Tensors**, unlike tf.constant.

## range

In [57]:
start = 3
limit = 18
delta = 3
res = tf.range(start, limit, delta)  # [3, 6, 9, 12, 15]

sess.run(res)

array([ 3,  6,  9, 12, 15], dtype=int32)

In [58]:
start = 3
limit = 1
delta = -0.5
res = tf.range(start, limit, delta)  # [3, 2.5, 2, 1.5]

sess.run(res)

array([3. , 2.5, 2. , 1.5], dtype=float32)

# placeholder and feed_dict

## placeholder

### compat.v1.placeholder

In [52]:
x = tf.compat.v1.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.compat.v1.Session() as sess:
    rand_array = np.random.rand(1024, 1024)
    print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.


[[253.62634 257.70514 249.42615 ... 255.15277 250.50844 247.60849]
 [264.37482 271.83875 261.26874 ... 265.66623 259.62592 262.2572 ]
 [244.46533 252.58687 238.42688 ... 246.29611 242.53079 243.05496]
 ...
 [251.85562 256.6626  238.39891 ... 254.60403 248.50131 244.00969]
 [252.5892  259.21112 245.94672 ... 252.2708  246.63446 249.92834]
 [257.60052 259.74472 254.56302 ... 259.0321  258.9993  259.6312 ]]


### placeholder

In [53]:
x = tf.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.Session() as sess:
    rand_array = np.random.rand(1024, 1024)
    print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.

[[262.22357 262.5691  245.57852 ... 253.93199 246.78749 263.85468]
 [263.06772 259.36884 252.49925 ... 261.35986 253.84753 261.98444]
 [262.67126 258.5251  253.73703 ... 251.98605 249.53722 260.8363 ]
 ...
 [270.4945  265.42654 251.63791 ... 258.32172 254.30774 262.84503]
 [266.57138 263.7236  251.74564 ... 253.52486 251.93059 261.6168 ]
 [254.34274 245.93073 241.51755 ... 242.95236 235.23454 245.96504]]
